In [ ]:
pip install jupyter_dash

In [ ]:
pip install dash_bootstrap_components

In [1]:
import pandas as pd
import numpy as np
import dash
from dash import Dash
from dash import jupyter_dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc
from dash import dash_table

In [2]:
df = pd.read_csv(r"C:\Users\joe\Josh-Guo-Data-Science\Foods Project\foods_df.csv")

In [ ]:
app = dash.Dash(external_stylesheets=[dbc.themes.LUMEN])
options_list = list(df.columns[1:])

target_column1 = "Ingredient" # This is for coloring the cuisine_ingredients_table DataTable
target_column2 = "Times Ingredient Was Used" # This is for coloring the cuisine_ingredients_usage DataTable

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Ingredients & Cuisines Dashboard", style={"font-size":60, "textAlign":"center"},),),
            ]),
    html.Br(),
    dbc.Row([dbc.Col(html.H1("Cuisine"), width="auto"
                    ),
            dbc.Col(
            dcc.Dropdown(
                id="cuisine_dropdown",
                options=[{'label': option, 'value':option} for option in list(df["cuisine"].unique())],
                searchable=True,
                placeholder="Select A Cuisine",
                        ), align="center"
                    )    
            ]),

    dbc.Row([dbc.Col(
            dash_table.DataTable(
                id="cuisine_ingredients_table",
                style_data_conditional=[{
                'if': {'column_id': target_column1},
                'backgroundColor': '#b3ff99',  
                                        }],
                style_data={ 'border': '2px solid black'},
                style_header={'fontWeight': 'bold'},
                                ), width=5
                    ),
             dbc.Col(
             dash_table.DataTable(
                 id="cuisine_ingredients_usage",
                 style_data_conditional=[{
                 'if': {'column_id': target_column2},
                 'backgroundColor': '#ff9933',  # Specify the background color
                                        }],
                 style_data={ 'border': '2px solid black'},
                 style_header={'fontWeight': 'bold'},
                                 ), width=5
                     )
            ], justify="evenly"),
    
    html.Br(),
    # Input for graph with most common cuisines for an ingredient
    dbc.Row([dbc.Col(html.H1("Ingredients"), width="auto"
                    ),
            dbc.Col(
            dcc.Dropdown(
                id="ingredient_dropdown",
                options=[{'label': option, 'value': option} for option in options_list],
                multi=True,
                searchable=True,
                placeholder="Select Any Ingredients",
                        ),
            width=8, align="center"
                    )
            ]),
    # Output for graph with most common cuisines for an ingredient
    dbc.Row([dbc.Col(
            dcc.Graph(id="ingredient_most_used_cuisine"),
                    )
            ])
])
@app.callback(Output(component_id="ingredient_most_used_cuisine", component_property="figure"),
              Input(component_id="ingredient_dropdown", component_property="value"))
def ingredient_top_cuisines(ingredient_list):
    ingredient_list.append("cuisine") # So the cuisine column is inlcluded
    ingredient_df = df[ingredient_list]
    # Using sum to get dishes with all the ingredients. The -1 accounts for "cuisine" being in the list
    ingredient_df = ingredient_df[ingredient_df.iloc[:, 0:-1].sum(axis=1) == len(ingredient_list) - 1]
    ingredient_df = pd.DataFrame(ingredient_df.groupby("cuisine")["cuisine"].count().sort_values(ascending=False))
    # Making the bar chart based on ingredient_df
    fig = px.histogram(ingredient_df, x=ingredient_df.index, y="cuisine", title="Most Common Cuisines for Chosen Ingredients",
                       labels={"index":"Cuisine"}, color_discrete_sequence=["#00d8d8"])
    fig.update_layout(xaxis_title="Cuisines", yaxis_title="Amount of Dishes")
    fig.for_each_trace(lambda t: t.update(hovertemplate=t.hovertemplate.replace("sum of cuisine", "Count")))
    fig.update_traces(marker_line_width=1.2, marker_line_color="black")
    return fig

@app.callback([Output(component_id="cuisine_ingredients_table", component_property="data"),
              Output(component_id="cuisine_ingredients_table", component_property="columns"),
              Input(component_id="cuisine_dropdown", component_property="value")])
def cuisine_top_ingredients(cuisine):
    ingredients = list(df.columns[1:])
    cuisine_ingredients = df[df["cuisine"] == cuisine]
    cuisine_ingredients = cuisine_ingredients[ingredients].sum(axis=0) # How many times each ingredient is used
    top = list(np.argsort(cuisine_ingredients.values)[-21:]) # Returns indexes of greatest values
    top_ingredients = cuisine_ingredients[top].sort_values(ascending=False) # This is a series
    top_ingredients_df = pd.DataFrame(top_ingredients).reset_index() # Converting it to a dataframe
    top_ingredients_df.rename(columns={"index":"Ingredient", 0: "Count"}, inplace=True)
    columns = [{"name": i, "id": i} for i in top_ingredients_df.columns]
    return top_ingredients_df.to_dict("records"), columns

@app.callback([Output(component_id="cuisine_ingredients_usage", component_property="data"),
               Output(component_id="cuisine_ingredients_usage", component_property="columns"),
               Input(component_id="cuisine_dropdown", component_property="value")])
def unique_ingredients(cuisine):
    unique_ingredients_list = [] # This list will take in the count of ingredients for each usage amount
    cuisine_dishes = df[df["cuisine"] == cuisine]
    
    for i in range(0, 105, 5):
        ingredients = cuisine_dishes.iloc[:, 1:].sum(axis=0) > i # Returns a series where each ingredient is True/False
        ingredients_count = ingredients.sum() # Summing the Trues gives the amount of ingredients passing the condition
        unique_ingredients_list.append(ingredients_count)
    
    # The dataframe made from this dictionary has the usage amount in one column and the count in the other
    unique_ingredients_dict = {"Times Ingredient Was Used": [str(num) + "+" for num in list(range(0, 105, 5))], "Count": unique_ingredients_list}
    unique_df = pd.DataFrame(unique_ingredients_dict)
    columns = [{"name": i, "id": i} for i in unique_df.columns]
    
    return unique_df.to_dict("records"), columns


if __name__ == "__main__":
    app.run_server(jupyter_mode="external")